In [1]:
from gc_utils import *

alg = Algebra(5)
ga2 = Algebra(2)
locals().update(alg.blades)

In [2]:
# 2d general linear transform
def random_linear_trans(alg: Algebra):
    matrix = np.random.rand(alg.d, alg.d)
    return matrix2trans(matrix, alg)

def random_ortho_trans(alg: Algebra):
    from scipy.stats import ortho_group
    matrix = ortho_group.rvs(dim=alg.d)
    return matrix2trans(matrix, alg)

f = random_linear_trans(ga2)
x = random_r_blade(1, ga2)
f(x)

0.634 𝐞₁ + 1.13 𝐞₂

In [3]:
# 2d polar decomposition
# f = SUU
df = derivative(f, x, ga2, grade=1)
UU = df / norm(df)
S = lambda x: UU * f(x)
S(x)*UU, f(x)

(0.634 𝐞₁ + 1.13 𝐞₂, 0.634 𝐞₁ + 1.13 𝐞₂)

In [4]:
# UU is rotor, S is symmetric
UU, curl(S, x, ga2, grade=1)

(0.935 + 0.354 𝐞₁₂, -1.11e-10 𝐞₁₂)

In [5]:
#6.3 remove the condition of 2 dimension
a, b, x = random_r_vectors(3, alg)
g = lambda x: x + (x|a)*b
abframe = [a,b]
r_frame = reciprocal(abframe)
dg = 2 + a*b
derivative(g, x, alg, grade=1, frame=abframe, r_frame=r_frame), dg

(2.92 + 0.0106 𝐞₁₂ + 0.0739 𝐞₁₃ + 0.11 𝐞₁₄ + -0.129 𝐞₁₅ + 0.092 𝐞₂₃ + 0.126 𝐞₂₄ + -0.215 𝐞₂₅ + -0.078 𝐞₃₄ + -0.382 𝐞₃₅ + -0.705 𝐞₄₅ + 3.48e-19 𝐞₁₂₃₄ + 2.46e-17 𝐞₁₂₃₅ + 4.52e-17 𝐞₁₂₄₅ + -3.89e-18 𝐞₁₃₄₅ + 1.33e-17 𝐞₂₃₄₅,
 2.92 + 0.0106 𝐞₁₂ + 0.0739 𝐞₁₃ + 0.11 𝐞₁₄ + -0.129 𝐞₁₅ + 0.092 𝐞₂₃ + 0.126 𝐞₂₄ + -0.215 𝐞₂₅ + -0.078 𝐞₃₄ + -0.382 𝐞₃₅ + -0.705 𝐞₄₅)

In [6]:
#6.3
dg_norm = norm(dg)
UU = dg / dg_norm
U = simple_rotor_sqrt(UU)
S = lambda x: U.sw(g(x))
curl(S, x, alg, grade=1), U.reverse().sw(S(x)), g(x)

(-1.11e-10 𝐞₁₂ + 3.89e-10 𝐞₁₃ + 5e-10 𝐞₁₄ + 3.33e-10 𝐞₁₅ + 2.78e-10 𝐞₂₃ + -2.78e-11 𝐞₂₄ + -5.55e-11 𝐞₂₅ + -1.94e-10 𝐞₃₄ + -1.11e-10 𝐞₃₅ + -1.67e-10 𝐞₄₅,
 1.02 𝐞₁ + 0.869 𝐞₂ + 0.751 𝐞₃ + 0.44 𝐞₄ + 0.242 𝐞₅ + 5.44e-19 𝐞₁₂₃₄₅,
 1.02 𝐞₁ + 0.869 𝐞₂ + 0.751 𝐞₃ + 0.44 𝐞₄ + 0.242 𝐞₅)

In [7]:
#6.6
frame = alg.frame
i, j = 1, 3
aij = (1/np.sqrt(2))*(frame[i] + frame[j])
[aij.sw(x) for x in frame]

[-1.0 𝐞₁, 1.0 𝐞₄, -1.0 𝐞₃, 1.0 𝐞₂, -1.0 𝐞₅]

In [8]:
#6.7
alpha = 3
Sij = lambda x: x + (alpha * (x|frame[i]) * frame[j])
[Sij(x) for x in frame]

[1 𝐞₁, 1 𝐞₂ + 3 𝐞₄, 1 𝐞₃, 1 𝐞₄, 1 𝐞₅]

In [9]:
#7.2
g = random_linear_trans(alg)
g_ = lambda x, y: x | g(y)
derivative(lambda x: g_(x, b), x, alg, grade=1), g(b)

(1.75 𝐞₁ + 1.98 𝐞₂ + 2.35 𝐞₃ + 1.18 𝐞₄ + 1.29 𝐞₅,
 1.75 𝐞₁ + 1.98 𝐞₂ + 2.35 𝐞₃ + 1.18 𝐞₄ + 1.29 𝐞₅)

In [10]:
# check nonsingular
outermorphism(g, alg.pseudoscalar((1,)), alg)

-0.0227 𝐞₁₂₃₄₅

In [11]:
#7.5 orthogonal transformation is its isometry
f = random_ortho_trans(alg)
x | f(b), f(x) | f(f(b))

(-0.257, -0.257)

In [12]:
# skew bilinear form
gm = skew_part(g, alg)
G = simplicial_derivative(lambda vectors: vectors[0] | gm(vectors[1]), np.zeros(2), alg)
x | gm(b), x | (b | G)

(0.564, 0.564)

In [13]:
#7.6b f is not an isometry of gm
adjoint_outermorphism(f, G, alg), G

(0.2 𝐞₁₂ + -0.54 𝐞₁₃ + -0.352 𝐞₁₄ + -0.0654 𝐞₁₅ + 0.0238 𝐞₂₃ + 0.0627 𝐞₂₄ + -0.000857 𝐞₂₅ + 0.116 𝐞₃₄ + -0.556 𝐞₃₅ + -0.0326 𝐞₄₅,
 0.303 𝐞₁₂ + 0.108 𝐞₁₃ + 0.0587 𝐞₁₄ + -0.156 𝐞₁₅ + 0.411 𝐞₂₃ + -0.393 𝐞₂₄ + -0.0766 𝐞₂₅ + -0.473 𝐞₃₄ + -0.322 𝐞₃₅ + -0.0167 𝐞₄₅)

In [14]:
#7.8 the inverse of gm
blades = bivector_split(G, alg)
gm(sum(-x | blade/normsq(blade) for blade in blades)), x

(0.711 𝐞₁ + 0.523 𝐞₂ + 0.71 𝐞₃ + 0.404 𝐞₄ + -0.152 𝐞₅,
 0.918 𝐞₁ + 0.717 𝐞₂ + 0.58 𝐞₃ + 0.0772 𝐞₄ + 0.0122 𝐞₅)

In [15]:
#7.12 assume splitted blades are normal
blades_norm = [norm(b) for b in blades]
blades = [b / n for b, n in zip(blades, blades_norm)]
gprod(blades)

3.66e-16 + -3.47e-18 𝐞₁₂ + -1.73e-18 𝐞₁₃ + 6.2e-17 𝐞₁₄ + 1.04e-17 𝐞₁₅ + 3.47e-18 𝐞₂₄ + 1.39e-17 𝐞₂₅ + 1.39e-17 𝐞₃₄ + -2.08e-17 𝐞₃₅ + -0.341 𝐞₁₂₃₄ + -0.68 𝐞₁₂₃₅ + 0.27 𝐞₁₂₄₅ + 0.403 𝐞₁₃₄₅ + -0.432 𝐞₂₃₄₅

In [16]:
#7.17
theta = np.random.rand(2)
rotors = [blade_exp(t*blade) for t, blade in zip(theta, blades)]
[(U.sw(G), G) for U in rotors for G in blades]

[(0.368 𝐞₁₂ + -0.297 𝐞₁₃ + -0.00999 𝐞₁₄ + -0.691 𝐞₁₅ + -0.0469 𝐞₂₃ + -0.0274 𝐞₂₄ + 0.374 𝐞₂₅ + 0.0208 𝐞₃₄ + -0.39 𝐞₃₅ + -0.0615 𝐞₄₅,
  0.368 𝐞₁₂ + -0.297 𝐞₁₃ + -0.0469 𝐞₂₃ + -0.00999 𝐞₁₄ + -0.0274 𝐞₂₄ + 0.0208 𝐞₃₄ + -0.691 𝐞₁₅ + 0.374 𝐞₂₅ + -0.39 𝐞₃₅ + -0.0615 𝐞₄₅),
 (0.242 𝐞₁₂ + 0.22 𝐞₁₃ + 0.0725 𝐞₁₄ + 0.0329 𝐞₁₅ + 0.5 𝐞₂₃ + -0.456 𝐞₂₄ + -0.208 𝐞₂₅ + -0.565 𝐞₃₄ + -0.258 𝐞₃₅ + -0.000319 𝐞₄₅,
  0.242 𝐞₁₂ + 0.22 𝐞₁₃ + 0.5 𝐞₂₃ + 0.0725 𝐞₁₄ + -0.456 𝐞₂₄ + -0.565 𝐞₃₄ + 0.0329 𝐞₁₅ + -0.208 𝐞₂₅ + -0.258 𝐞₃₅ + -0.000319 𝐞₄₅),
 (0.368 𝐞₁₂ + -0.297 𝐞₁₃ + -0.00999 𝐞₁₄ + -0.691 𝐞₁₅ + -0.0469 𝐞₂₃ + -0.0274 𝐞₂₄ + 0.374 𝐞₂₅ + 0.0208 𝐞₃₄ + -0.39 𝐞₃₅ + -0.0615 𝐞₄₅,
  0.368 𝐞₁₂ + -0.297 𝐞₁₃ + -0.0469 𝐞₂₃ + -0.00999 𝐞₁₄ + -0.0274 𝐞₂₄ + 0.0208 𝐞₃₄ + -0.691 𝐞₁₅ + 0.374 𝐞₂₅ + -0.39 𝐞₃₅ + -0.0615 𝐞₄₅),
 (0.242 𝐞₁₂ + 0.22 𝐞₁₃ + 0.0725 𝐞₁₄ + 0.0329 𝐞₁₅ + 0.5 𝐞₂₃ + -0.456 𝐞₂₄ + -0.208 𝐞₂₅ + -0.565 𝐞₃₄ + -0.258 𝐞₃₅ + -0.000319 𝐞₄₅,
  0.242 𝐞₁₂ + 0.22 𝐞₁₃ + 0.5 𝐞₂₃ + 0.0725 𝐞₁₄ + -0.456 𝐞₂₄ + -0.565 𝐞₃₄ + 0.0329 

In [17]:
#7.18
first_vecs = [blade_split(blade, alg)[0] for blade in blades]
first_vecs = [v/norm(v) for v in first_vecs]
second_vecs = [v | blade for v, blade in zip(first_vecs, blades)]
inter_plane_rotors = [simple_rotor_sqrt(a * b) for a, b in (first_vecs, second_vecs)]
U = gprod(inter_plane_rotors)
terms_ratio(U.sw(blades[0]), blades[1])

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [18]:
f"{x[1:]}"

'(0.7171633963229438, 0.5799614201709699, 0.077164520130231, 0.012189256628512002)'

In [19]:
G_normalized = sum(blades)
G_normalized, U.sw(G_normalized)

(0.61 𝐞₁₂ + -0.0765 𝐞₁₃ + 0.0625 𝐞₁₄ + -0.658 𝐞₁₅ + 0.453 𝐞₂₃ + -0.483 𝐞₂₄ + 0.166 𝐞₂₅ + -0.545 𝐞₃₄ + -0.648 𝐞₃₅ + -0.0618 𝐞₄₅,
 0.61 𝐞₁₂ + -0.0765 𝐞₁₃ + 0.0625 𝐞₁₄ + -0.658 𝐞₁₅ + 0.453 𝐞₂₃ + -0.483 𝐞₂₄ + 0.166 𝐞₂₅ + -0.545 𝐞₃₄ + -0.648 𝐞₃₅ + -0.0618 𝐞₄₅)

In [20]:
# we can recover this "normalization"
sum(P(G_normalized, b) * n for b, n in zip(blades, blades_norm)), G

(0.303 𝐞₁₂ + 0.108 𝐞₁₃ + 0.0587 𝐞₁₄ + -0.156 𝐞₁₅ + 0.411 𝐞₂₃ + -0.393 𝐞₂₄ + -0.0766 𝐞₂₅ + -0.473 𝐞₃₄ + -0.322 𝐞₃₅ + -0.0167 𝐞₄₅,
 0.303 𝐞₁₂ + 0.108 𝐞₁₃ + 0.0587 𝐞₁₄ + -0.156 𝐞₁₅ + 0.411 𝐞₂₃ + -0.393 𝐞₂₄ + -0.0766 𝐞₂₅ + -0.473 𝐞₃₄ + -0.322 𝐞₃₅ + -0.0167 𝐞₄₅)

In [21]:
#7.21 zero if not onto the decomposed blades 
sympletic_basis = first_vecs + second_vecs
list(zip((v | (w | G_normalized) for v, w in product(sympletic_basis, repeat=2)), (product(['f', 'fp', 's', 'sp'], repeat=2))))

[(5.55e-17, ('f', 'f')),
 (-5.55e-17, ('f', 'fp')),
 (-1.0, ('f', 's')),
 (-2.08e-17, ('f', 'sp')),
 (8.33e-17, ('fp', 'f')),
 (-6.94e-18, ('fp', 'fp')),
 (8.33e-17, ('fp', 's')),
 (-1.0, ('fp', 'sp')),
 (1.0, ('s', 'f')),
 (-1.11e-16, ('s', 'fp')),
 (-5.55e-17, ('s', 's')),
 (-1.11e-16, ('s', 'sp')),
 (, ('sp', 'f')),
 (1.0, ('sp', 'fp')),
 (9.71e-17, ('sp', 's')),
 (1.39e-17, ('sp', 'sp'))]

In [22]:
# Recover the normalization
[gm(v)/n for v, n in zip(sympletic_basis, blades_norm*2)], sympletic_basis

([1.04e-16 𝐞₁ + -0.44 𝐞₂ + 0.355 𝐞₃ + 0.0119 𝐞₄ + 0.825 𝐞₅,
  -0.323 𝐞₁ + 0.668 𝐞₃ + -0.61 𝐞₄ + -0.278 𝐞₅,
  0.837 𝐞₁ + -0.292 𝐞₂ + 0.342 𝐞₃ + 0.0702 𝐞₄ + -0.304 𝐞₅,
  -0.0939 𝐞₁ + -0.748 𝐞₂ + -0.488 𝐞₃ + -0.401 𝐞₄ + -0.183 𝐞₅],
 [-0.837 𝐞₁ + 0.292 𝐞₂ + -0.342 𝐞₃ + -0.0702 𝐞₄ + 0.304 𝐞₅,
  0.0939 𝐞₁ + 0.748 𝐞₂ + 0.488 𝐞₃ + 0.401 𝐞₄ + 0.183 𝐞₅,
  -2.78e-17 𝐞₁ + -0.44 𝐞₂ + 0.355 𝐞₃ + 0.0119 𝐞₄ + 0.825 𝐞₅,
  -0.323 𝐞₁ + 6.94e-18 𝐞₂ + 0.668 𝐞₃ + -0.61 𝐞₄ + -0.278 𝐞₅])

In [23]:
#7.26a symplectic transformations of gm (isometries of skew bilinear form)
# conditions: unity and sympletic basis
translation = lambda x: (
    x + 
    2*(x|second_vecs[1]) * first_vecs[1] + 
    3*(x|second_vecs[0]) * first_vecs[0]
    )

outermorphism(translation, G_normalized, alg), G_normalized, outermorphism(translation, G, alg), G

(0.61 𝐞₁₂ + -0.0765 𝐞₁₃ + 0.0625 𝐞₁₄ + -0.658 𝐞₁₅ + 0.453 𝐞₂₃ + -0.483 𝐞₂₄ + 0.166 𝐞₂₅ + -0.545 𝐞₃₄ + -0.648 𝐞₃₅ + -0.0618 𝐞₄₅,
 0.61 𝐞₁₂ + -0.0765 𝐞₁₃ + 0.0625 𝐞₁₄ + -0.658 𝐞₁₅ + 0.453 𝐞₂₃ + -0.483 𝐞₂₄ + 0.166 𝐞₂₅ + -0.545 𝐞₃₄ + -0.648 𝐞₃₅ + -0.0618 𝐞₄₅,
 0.303 𝐞₁₂ + 0.108 𝐞₁₃ + 0.0587 𝐞₁₄ + -0.156 𝐞₁₅ + 0.411 𝐞₂₃ + -0.393 𝐞₂₄ + -0.0766 𝐞₂₅ + -0.473 𝐞₃₄ + -0.322 𝐞₃₅ + -0.0167 𝐞₄₅,
 0.303 𝐞₁₂ + 0.108 𝐞₁₃ + 0.0587 𝐞₁₄ + -0.156 𝐞₁₅ + 0.411 𝐞₂₃ + -0.393 𝐞₂₄ + -0.0766 𝐞₂₅ + -0.473 𝐞₃₄ + -0.322 𝐞₃₅ + -0.0167 𝐞₄₅)

In [24]:
#7.26b
rotation = lambda x: (
    x + 
    4*((x|first_vecs[1]) * second_vecs[1] + (x|second_vecs[0]) * first_vecs[0])
    )

outermorphism(rotation, G_normalized, alg), G_normalized, outermorphism(rotation, G, alg), G

(0.61 𝐞₁₂ + -0.0765 𝐞₁₃ + 0.0625 𝐞₁₄ + -0.658 𝐞₁₅ + 0.453 𝐞₂₃ + -0.483 𝐞₂₄ + 0.166 𝐞₂₅ + -0.545 𝐞₃₄ + -0.648 𝐞₃₅ + -0.0618 𝐞₄₅,
 0.61 𝐞₁₂ + -0.0765 𝐞₁₃ + 0.0625 𝐞₁₄ + -0.658 𝐞₁₅ + 0.453 𝐞₂₃ + -0.483 𝐞₂₄ + 0.166 𝐞₂₅ + -0.545 𝐞₃₄ + -0.648 𝐞₃₅ + -0.0618 𝐞₄₅,
 0.303 𝐞₁₂ + 0.108 𝐞₁₃ + 0.0587 𝐞₁₄ + -0.156 𝐞₁₅ + 0.411 𝐞₂₃ + -0.393 𝐞₂₄ + -0.0766 𝐞₂₅ + -0.473 𝐞₃₄ + -0.322 𝐞₃₅ + -0.0167 𝐞₄₅,
 0.303 𝐞₁₂ + 0.108 𝐞₁₃ + 0.0587 𝐞₁₄ + -0.156 𝐞₁₅ + 0.411 𝐞₂₃ + -0.393 𝐞₂₄ + -0.0766 𝐞₂₅ + -0.473 𝐞₃₄ + -0.322 𝐞₃₅ + -0.0167 𝐞₄₅)

In [25]:
#7.26c
outermorphism(lambda x: rotation(translation(x)), G, alg), G

(0.303 𝐞₁₂ + 0.108 𝐞₁₃ + 0.0587 𝐞₁₄ + -0.156 𝐞₁₅ + 0.411 𝐞₂₃ + -0.393 𝐞₂₄ + -0.0766 𝐞₂₅ + -0.473 𝐞₃₄ + -0.322 𝐞₃₅ + -0.0167 𝐞₄₅,
 0.303 𝐞₁₂ + 0.108 𝐞₁₃ + 0.0587 𝐞₁₄ + -0.156 𝐞₁₅ + 0.411 𝐞₂₃ + -0.393 𝐞₂₄ + -0.0766 𝐞₂₅ + -0.473 𝐞₃₄ + -0.322 𝐞₃₅ + -0.0167 𝐞₄₅)

In [26]:
#7.28
gp = sym_part(g, alg)
matrix = trans2matrix(gp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
neg_vecs, pos_vecs, null_vecs = [], [], []
for e, v in zip(eigvalues, eigvecs):
    if e < -1e-6:
        neg_vecs.append(v)
    elif e > 1e-6:
        pos_vecs.append(v)
    else:
        null_vecs.append(v)

neg_blade = wedge(neg_vecs)
nv_sign = (-1) ** len(neg_vecs) 
c = lambda x: nv_sign*neg_blade.sw(x)
h = lambda x: sum(np.sqrt(abs(e)) * P(x, v) for e, v in zip(eigvalues, eigvecs))

x | gp(b), h(x)|c(h(b)), c(x)|h(h(b))

(3.93,
 3.93 + -4.29e-18 𝐞₁₂₃₄ + 4.46e-18 𝐞₁₂₃₅ + -6.81e-18 𝐞₁₂₄₅ + 7.83e-18 𝐞₁₃₄₅ + -9.9e-18 𝐞₂₃₄₅,
 3.93 + 7.57e-17 𝐞₁₂₃₄ + -8.14e-17 𝐞₁₂₃₅ + 7.47e-17 𝐞₁₂₄₅ + -6.98e-17 𝐞₁₃₄₅ + 6.79e-17 𝐞₂₃₄₅)

In [73]:
#7.28 If the symmetric transformation is positive definite (Euclidean metric)
gpp = lambda x: sum(abs(e) * P(x, v) for e, v in zip(eigvalues, eigvecs))
hinv = lambda x: sum(1/np.sqrt(abs(e)) * P(x, v) for e, v in zip(eigvalues, eigvecs))
r = 3 # a arbitrary integer <= n
O = normalize(random_versor(r, alg))
f = lambda x: -hinv(O.sw(h(x)))
x | gpp(b), f(x) | gpp(f(b)), x, f(x)

(3.58,
 3.58,
 0.918 𝐞₁ + 0.717 𝐞₂ + 0.58 𝐞₃ + 0.0772 𝐞₄ + 0.0122 𝐞₅,
 -0.826 𝐞₁ + -1.11 𝐞₂ + 3.13 𝐞₃ + -1.29 𝐞₄ + 0.0282 𝐞₅)

In [74]:
# But if gp is not positive definite, the orthogonal transform is not an isometry
f = lambda x: hinv(O.sw(h(x)))
x | gp(b), f(x) | gp(f(b)), f(x)

(3.93,
 1.23 + -6.61e-33 𝐞₁₂ + -3.42e-33 𝐞₁₃ + -2.71e-33 𝐞₁₄ + -2.91e-33 𝐞₁₅ + 4.77e-33 𝐞₂₃ + 7.47e-33 𝐞₂₄ + 6.06e-33 𝐞₂₅ + 1.91e-33 𝐞₃₄ + 9.89e-34 𝐞₃₅ + -9.25e-34 𝐞₄₅,
 0.826 𝐞₁ + 1.11 𝐞₂ + -3.13 𝐞₃ + 1.29 𝐞₄ + -0.0282 𝐞₅)

In [78]:
#7.33 Signature: the inner product under metric c splits into two part, according to neg_blade
def ortho_split(x, A):
    xp = P(x, A)
    return xp, x-xp

xp, xo = ortho_split(x, neg_blade)
bp, bo = ortho_split(b, neg_blade)
neg_blade, x | c(b), (xo | bo) - (xp | bp)


(0.488 𝐞₁₂ + 0.132 𝐞₁₃ + -0.0164 𝐞₁₄ + -0.603 𝐞₁₅ + 0.202 𝐞₂₃ + 0.327 𝐞₂₄ + -0.0223 𝐞₂₅ + 0.0951 𝐞₃₄ + 0.243 𝐞₃₅ + 0.405 𝐞₄₅,
 1.5 + -4.49e-20 𝐞₁₂₃₄ + 2.84e-19 𝐞₁₂₃₅ + -2.14e-18 𝐞₁₂₄₅ + 2.64e-18 𝐞₁₃₄₅ + -3.39e-18 𝐞₂₃₄₅,
 1.5)

In [83]:
# If R is a rotor (see next section) preserves the reflected subspace (neg_blade)
# Then R is an isometry
R = 1
if len(neg_vecs) >= 2:
    R *= blade_exp(3.3 * wedge(neg_vecs[:2]))
if len(pos_vecs) >= 2:
    R *= blade_exp(5.5 * wedge(pos_vecs[-2:]))
f = lambda x: hinv(R.sw(h(x)))

R.sw(neg_blade), neg_blade, x | gp(b), f(x) | gp(f(b))

(0.488 𝐞₁₂ + 0.132 𝐞₁₃ + -0.0164 𝐞₁₄ + -0.603 𝐞₁₅ + 0.202 𝐞₂₃ + 0.327 𝐞₂₄ + -0.0223 𝐞₂₅ + 0.0951 𝐞₃₄ + 0.243 𝐞₃₅ + 0.405 𝐞₄₅,
 0.488 𝐞₁₂ + 0.132 𝐞₁₃ + -0.0164 𝐞₁₄ + -0.603 𝐞₁₅ + 0.202 𝐞₂₃ + 0.327 𝐞₂₄ + -0.0223 𝐞₂₅ + 0.0951 𝐞₃₄ + 0.243 𝐞₃₅ + 0.405 𝐞₄₅,
 3.93,
 3.93 + 3.49e-33 𝐞₁₂ + 1.13e-33 𝐞₁₃ + -2.49e-33 𝐞₁₄ + -6.08e-33 𝐞₁₅ + 3.14e-33 𝐞₂₃ + -1.25e-33 𝐞₂₄ + 1.07e-33 𝐞₂₅ + -2.79e-33 𝐞₃₄ + 2.48e-33 𝐞₃₅ + 2.63e-33 𝐞₄₅)

In [87]:
# If R is a spinor preserves the reflected subspace (neg_blade)
# Then R is an isometry
R = 1
if len(neg_vecs) >= 2:
    R *= blade_exp(3.3 * wedge(neg_vecs[:2]))
if len(pos_vecs) >= 2:
    R *= blade_exp(5.5 * wedge(pos_vecs[:2]))
R *= pos_vecs[-1]
R *= neg_vecs[-2]
f = lambda x: hinv(R.sw(h(x)))

R.sw(neg_blade), neg_blade, x | gp(b), f(x) | gp(f(b))

(-0.488 𝐞₁₂ + -0.132 𝐞₁₃ + 0.0164 𝐞₁₄ + 0.603 𝐞₁₅ + -0.202 𝐞₂₃ + -0.327 𝐞₂₄ + 0.0223 𝐞₂₅ + -0.0951 𝐞₃₄ + -0.243 𝐞₃₅ + -0.405 𝐞₄₅,
 0.488 𝐞₁₂ + 0.132 𝐞₁₃ + -0.0164 𝐞₁₄ + -0.603 𝐞₁₅ + 0.202 𝐞₂₃ + 0.327 𝐞₂₄ + -0.0223 𝐞₂₅ + 0.0951 𝐞₃₄ + 0.243 𝐞₃₅ + 0.405 𝐞₄₅,
 3.93,
 3.93 + 2.43e-35 𝐞₁₂ + -4.27e-34 𝐞₁₃ + -8.51e-35 𝐞₁₄ + 5.52e-35 𝐞₁₅ + 1.38e-33 𝐞₂₃ + 1.21e-34 𝐞₂₄ + -9.38e-35 𝐞₂₅ + 1.86e-35 𝐞₃₄ + -4.28e-34 𝐞₃₅ + -1.14e-34 𝐞₄₅)

In [32]:
# From the next section O(p, q) is a versor form Algebra(p, q)
# If we choose a versor from eigenvectors
R = wedge([eigvecs[i] for i in (0,2,3)]) # eigenvectors are orthogonal
R = (R / norm(R))
f = lambda x: hinv(R.sw(h(x)))
x | gp(b), f(x) | gp(f(b)), f(x)

(3.93, 3.93, 0.124 𝐞₁ + -0.597 𝐞₂ + -0.517 𝐞₃ + -0.945 𝐞₄ + -0.41 𝐞₅)

In [85]:
# From the next section O(p, q) is a versor form Algebra(p, q)
# If we choose a versor from eigenvectors
# FIXME some mystery here
B = (pos_vecs[0] ^ neg_vecs[1])
R = blade_exp(3.3*B)
f = lambda x: hinv(R.sw(h(x)))
x | gp(b), f(x) | gp(f(b)), pos_vecs[0] | neg_vecs[1]

(3.93,
 3.95 + 1.81e-34 𝐞₁₂ + 1.02e-35 𝐞₁₃ + -6.68e-35 𝐞₁₄ + -1.93e-34 𝐞₁₅ + 1.71e-34 𝐞₂₃ + 3.55e-35 𝐞₂₄ + 6.77e-35 𝐞₂₅ + -1.31e-34 𝐞₃₄ + 3.55e-34 𝐞₃₅ + -1.87e-34 𝐞₄₅,
 3.19e-16)

In [46]:
f = lambda x: hinv(B.sw(h(x)))
x | gp(b), f(x) | gp(f(b)), f(x)

(3.93,
 3.93 + 2.46e-33 𝐞₁₂ + -1.56e-34 𝐞₁₃ + -8.95e-34 𝐞₁₄ + 1.24e-35 𝐞₁₅ + 3.99e-33 𝐞₂₃ + -2.09e-33 𝐞₂₄ + -1.68e-33 𝐞₂₅ + 1.44e-33 𝐞₃₄ + 5.96e-34 𝐞₃₅ + 3.02e-34 𝐞₄₅,
 -0.0173 𝐞₁ + 0.41 𝐞₂ + 0.643 𝐞₃ + 1.02 𝐞₄ + 0.284 𝐞₅)

In [35]:
signature = (len(pos_vecs), len(neg_vecs))
pseudo = Algebra(*signature)
O = random_versor(3, pseudo)
O = O / norm(O)
signature

(3, 2)

In [47]:
blade_names = pseudo.blades.blades.keys()
def parse_e_string(e_string):
    # Extract the numeric part of the string and convert each character to an integer
    return tuple(int(char) - 1 for char in e_string[1:])

[parse_e_string(k) for k in blade_names]

[(),
 (0,),
 (1,),
 (2,),
 (3,),
 (4,),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4),
 (0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 2, 3),
 (0, 2, 4),
 (0, 3, 4),
 (1, 2, 3),
 (1, 2, 4),
 (1, 3, 4),
 (2, 3, 4),
 (0, 1, 2, 3),
 (0, 1, 2, 4),
 (0, 1, 3, 4),
 (0, 2, 3, 4),
 (1, 2, 3, 4),
 (0, 1, 2, 3, 4)]

In [48]:
R = 0
bases = pos_vecs + neg_vecs
for key in blade_names:
    indices = parse_e_string(key)
    mv = wedge([bases[i] for i in indices])
    R += O[key] * mv
R = R / norm(R)
f = lambda x: hinv(R.sw(h(x)))
x | gp(b), f(x) | gp(f(b)), f(x)

(3.93,
 -1.66 + -5.76e-34 𝐞₁₂ + 9.35e-34 𝐞₁₃ + -5.01e-34 𝐞₁₄ + -5.11e-33 𝐞₁₅ + -1.23e-34 𝐞₂₃ + -1.64e-33 𝐞₂₄ + 4.32e-33 𝐞₂₅ + 2.71e-33 𝐞₃₄ + -1.66e-33 𝐞₃₅ + 3.22e-33 𝐞₄₅,
 -0.108 𝐞₁ + 1.65 𝐞₂ + -0.866 𝐞₃ + 1.88 𝐞₄ + -2.89 𝐞₅ + 5.06e-34 𝐞₁₂₃ + 4.16e-34 𝐞₁₂₄ + -3.77e-33 𝐞₁₂₅ + -1.08e-33 𝐞₁₃₄ + -6.43e-34 𝐞₁₃₅ + 1.11e-33 𝐞₁₄₅ + -1.33e-33 𝐞₂₃₄ + 1.39e-33 𝐞₂₃₅ + -1.97e-33 𝐞₂₄₅ + -1.86e-33 𝐞₃₄₅)

So a rotation preserves the reflected subspace is an isometry. Any other isometry? The above we split the space into pos_vecs and neg_vecs. The metric's signature encode such split. The next section we encode the metric into the algebra. So an orthogonal transform becomes an isometry of the vector space.

In [37]:
# signature
[v | c(v) for v in eigvecs]

[-1.0 + 9.01e-18 𝐞₁₂₃₄ + 5.43e-18 𝐞₁₂₃₅ + 3.46e-18 𝐞₁₂₄₅ + -4.47e-18 𝐞₁₃₄₅ + -7.07e-18 𝐞₂₃₄₅,
 -1.0,
 1.0 + 1.17e-17 𝐞₁₂₃₄ + 3.69e-18 𝐞₁₂₃₅ + -4.15e-18 𝐞₁₂₄₅ + 1.51e-17 𝐞₁₃₄₅ + 6.19e-18 𝐞₂₃₄₅,
 1.0 + 6.45e-18 𝐞₁₂₃₄ + -2.19e-17 𝐞₁₂₃₅ + -5.22e-18 𝐞₁₂₄₅ + 4.71e-18 𝐞₁₃₄₅ + -1.41e-17 𝐞₂₃₄₅,
 1.0]

In [38]:
p, q, r = 0,0,0
for e in eigvalues:
    if e > 1e-6:
        p += 1
    elif e < -1e-6:
        q += 1
    else:
        r += 1
p, q, r

(3, 2, 0)

In [39]:
#8.2 Say the reflected subspace is 1d
alg = Algebra(3, 1)
locals().update(alg.blades)
x, y, u = random_r_vectors(3, alg)
u = u / norm(u)
R = lambda x: - u.sw(x)
R(x) | R(y), x | y

(0.8, 0.8)

Cartan-Dieudonne theorem: An isometry is a composition of at most d reflections. Identifying invariants of isometries as elements of geometry leads to modern developments.

In [40]:
#8.3 Cartan-Dieudonne theorem: An isometry is a composition of at most d reflections.

k = 3 # k <= d
vectors = random_r_vectors(k, alg)
p = sum((v**2)[0] > 0 for v in vectors)
U = gprod(vectors) # a versor, of either even or odd grades
f = lambda x: (-1) ** k * U * x / U
f_ = lambda x: (-1) ** p * U.sw(x) / normsq(U) #8.8b
f(x) | f(y), x | y, f_(x) | f_(y)

(0.8 + -2.38e-17 𝐞₁₂ + 7.55e-17 𝐞₁₃ + -6.38e-17 𝐞₁₄ + 4.69e-17 𝐞₂₃ + 2.79e-17 𝐞₂₄ + 6.64e-18 𝐞₃₄,
 0.8,
 0.8)

In [41]:
#8.8
def random_versor(r, alg):
    return gprod(random_r_vectors(r, alg))

def random_versor_(r, alg):
    vectors = random_r_vectors(r, alg)
    p = sum((v**2)[0] > 0 for v in vectors)
    return gprod(vectors), p
k = 3
for _ in range(10):
    U, p = random_versor_(k, alg)
    U = U/norm(U)
    print(k - p, U.reverse() * U)

1 -1.0 + 3.55e-15 𝐞₁₃ + -7.55e-15 𝐞₁₄ + -1.78e-15 𝐞₃₄ + -3.55e-15 𝐞₁₂₃₄
0 1.0 + -2.78e-17 𝐞₁₃ + -9.71e-17 𝐞₁₄ + -5.55e-17 𝐞₂₃ + -3.47e-17 𝐞₂₄ + -5.55e-17 𝐞₃₄ + -1.11e-16 𝐞₁₂₃₄
1 -1.0 + 1.11e-16 𝐞₁₃ + -3.33e-16 𝐞₁₄ + -4.16e-17 𝐞₂₃ + 8.88e-16 𝐞₂₄ + 1.11e-16 𝐞₃₄ + 2.22e-16 𝐞₁₂₃₄
0 1.0 + -2.43e-17 𝐞₁₃ + 6.94e-18 𝐞₁₄ + 8.33e-17 𝐞₂₃ + -2.78e-17 𝐞₃₄ + -8.33e-17 𝐞₁₂₃₄
0 1.0 + -2.78e-17 𝐞₁₂ + 4.34e-18 𝐞₁₃ + -1.11e-16 𝐞₁₄ + -5.2e-18 𝐞₂₃ + 6.61e-18 𝐞₃₄ + 2.36e-16 𝐞₁₂₃₄
0 1.0 + 5.2e-18 𝐞₁₂ + -1.11e-16 𝐞₂₄ + 1.67e-16 𝐞₁₂₃₄
0 1.0 + 4.44e-16 𝐞₃₄ + 1.78e-15 𝐞₁₂₃₄
0 1.0 + 4.86e-17 𝐞₁₃ + -3.82e-17 𝐞₁₄ + -4.16e-17 𝐞₂₃ + -1.39e-17 𝐞₂₄ + 1.39e-17 𝐞₃₄ + -1.46e-16 𝐞₁₂₃₄
0 1.0 + -3.47e-18 𝐞₁₂ + -1.3e-17 𝐞₁₃ + -1.56e-17 𝐞₁₄ + 8.67e-19 𝐞₂₃ + -4.99e-18 𝐞₂₄ + 3.04e-18 𝐞₁₂₃₄
1 -1.0 + -1.25e-15 𝐞₁₃ + -6.01e-15 𝐞₁₄ + -2.66e-15 𝐞₂₄ + 1.55e-15 𝐞₃₄ + 1.47e-14 𝐞₁₂₃₄


In [42]:
# versors (Pin) are 2:1 homomorphisc to orthogonal transformations (O)
U = random_versor(3, alg)
U.sw(x), (-U).sw(x)

(0.00737 𝐞₁ + -0.00487 𝐞₂ + -0.00378 𝐞₃ + -0.00156 𝐞₄,
 0.00737 𝐞₁ + -0.00487 𝐞₂ + -0.00378 𝐞₃ + -0.00156 𝐞₄)

In [43]:
#8.5
I = alg.pseudoscalar((1,))
I.sw(x), x

(0.41 𝐞₁ + 0.159 𝐞₂ + 0.669 𝐞₃ + 0.0017 𝐞₄,
 0.41 𝐞₁ + 0.159 𝐞₂ + 0.669 𝐞₃ + 0.0017 𝐞₄)

In [44]:
#8.7 orthogonal transformations (O) with determinant 1 (SO) correspond to even versors (Spin)
for k in range(1,5):
    U = gprod(random_r_vectors(k, alg)) # a versor, of either even or odd grades
    f = lambda x: (-1) ** (k*alg.d) * U.sw(x)/normsq(U)
    print(k, outermorphism(f, I, alg))

1 -1.0 𝐞₁₂₃₄
2 1.0 𝐞₁₂₃₄
3 -1.0 𝐞₁₂₃₄
4 1.0 𝐞₁₂₃₄


In [45]:
#8.9 rotors (Spin+) are a part of even versors (Spin), correpond to a part of SO as rotations (SO+)
def random_even_versor(alg):
    d = alg.d
    if (alg.d % 2):
        d -= 1
    return gprod(random_r_vectors(d, alg))


def random_rotor(alg, max_iter=100):
    for _ in range(max_iter):
        S = random_even_versor(alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)
        

def random_rotor_count(alg, max_iter=100):
    count = 0
    for _ in range(max_iter):
        S = random_even_versor(alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            count += 1
    return count

[(p, q, r, random_rotor_count(Algebra(p, q, r), 2000)/2000) for p, q, r in ((3,1,0),(1,3,0),(4,1,0),(3,0,1),(2,2,0))]

[(3, 1, 0, 0.6415),
 (1, 3, 0, 0.672),
 (4, 1, 0, 0.8055),
 (3, 0, 1, 1.0),
 (2, 2, 0, 0.492)]

versors (Pin) are products of {1, a, b, ab} and a rotor

In [46]:
#8.12
def random_simple_rotor(alg):
    while True:
        S = random_versor(2, alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)

In [47]:
#8.13
S = random_simple_rotor(alg)
u = x | S.grade(2)
u = u/norm(u)
v = S.sw(u)
v, S ** 2 * u, v ** 2, u ** 2, v * u, S ** 2

(0.598 𝐞₁ + 0.395 𝐞₂ + 1.06 𝐞₃ + 0.798 𝐞₄,
 0.598 𝐞₁ + 0.395 𝐞₂ + 1.06 𝐞₃ + 0.798 𝐞₄ + 5.75e-17 𝐞₁₂₃ + 1.4e-16 𝐞₁₂₄ + 3.26e-17 𝐞₁₃₄ + 2.6e-18 𝐞₂₃₄,
 1.0,
 1.0,
 0.02 + -0.119 𝐞₁₂ + -1.15 𝐞₁₃ + -0.63 𝐞₁₄ + -0.549 𝐞₂₃ + -0.258 𝐞₂₄ + 0.418 𝐞₃₄,
 0.02 + -0.119 𝐞₁₂ + -1.15 𝐞₁₃ + -0.63 𝐞₁₄ + -0.549 𝐞₂₃ + -0.258 𝐞₂₄ + 0.418 𝐞₃₄ + 1.04e-16 𝐞₁₂₃₄)

In [48]:
S ** 2 * (S ** 2).reverse(), S * S.reverse()

(1.0 + -2.78e-17 𝐞₁₂ + -2.68e-17 𝐞₁₃ + -1.61e-18 𝐞₁₄ + -3.78e-18 𝐞₂₃ + 8.68e-18 𝐞₂₄ + -1.93e-17 𝐞₃₄ + -1.99e-16 𝐞₁₂₃₄,
 1.0 + 5.55e-17 𝐞₁₃ + -2.78e-17 𝐞₁₄ + 1.39e-17 𝐞₂₄ + -1.04e-16 𝐞₁₂₃₄)

In [49]:
#8.15
(S ** 2).reverse(), u * v, v * u

(0.02 + 0.119 𝐞₁₂ + 1.15 𝐞₁₃ + 0.63 𝐞₁₄ + 0.549 𝐞₂₃ + 0.258 𝐞₂₄ + -0.418 𝐞₃₄ + 1.04e-16 𝐞₁₂₃₄,
 0.02 + 0.119 𝐞₁₂ + 1.15 𝐞₁₃ + 0.63 𝐞₁₄ + 0.549 𝐞₂₃ + 0.258 𝐞₂₄ + -0.418 𝐞₃₄,
 0.02 + -0.119 𝐞₁₂ + -1.15 𝐞₁₃ + -0.63 𝐞₁₄ + -0.549 𝐞₂₃ + -0.258 𝐞₂₄ + 0.418 𝐞₃₄)

In [50]:
#8.16
S2 = S ** 2
blade = S2.grade(2)
bladesq = (blade ** 2)
bladesq

-1.0 + 2.78e-17 𝐞₁₂ + 2.01e-16 𝐞₁₂₃₄

In [51]:
# 3 types of rotors (Spin+)
def type_convert(n):
    if n > 1e-6:
        return 1
    elif n < -1e-6:
        return -1
    else:
        return 0

# Unfortunately parabolic rotors are of measure zero, 
# so this function has zero chance generating them
def random_simple_rotor(alg, signature=None, max_iter=100):
    for _ in range(max_iter):
        S = random_versor(2, alg)
        if signature is not None:
            if type_convert((S.grade(2) ** 2)[0]) != signature:
                continue
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)
        
[random_simple_rotor(alg, signature=i) for i in range(-1, 2)]

[0.842 + 0.147 𝐞₁₂ + 0.18 𝐞₁₃ + 0.119 𝐞₁₄ + 0.623 𝐞₂₃ + 0.128 𝐞₂₄ + -0.347 𝐞₃₄,
 None,
 1.17 + 0.999 𝐞₁₂ + -0.298 𝐞₁₃ + 1.07 𝐞₁₄ + -1.04 𝐞₂₃ + -0.164 𝐞₂₄ + 1.17 𝐞₃₄]

In [52]:
def simple_rotor_log(R: MultiVector, tol=1e-6):
    blade = R.grade(2)
    signature = (blade ** 2)[0]
    if signature > tol:
        return np.arccosh(R[0]) / norm(blade) * blade
    if signature < -tol:
        return np.arccos(R[0]) / norm(blade) * blade
    else:
        return blade

In [53]:
#8.17a elliptic rotor (rotation), run several times you can see sqrt allows a sign change
S = random_simple_rotor(alg, signature=-1)
S2 = S ** 2
b = simple_rotor_log(S2)
S2, blade_exp(b), blade_exp(b - 6*np.pi*b/norm(b)), S, blade_exp(b/2), simple_rotor_sqrt(S2), blade_exp(3.3*b).sw(S)

(0.385 + 0.604 𝐞₁₂ + -0.613 𝐞₁₃ + -0.134 𝐞₁₄ + -0.582 𝐞₂₃ + -0.38 𝐞₂₄ + 0.256 𝐞₃₄ + -3.47e-17 𝐞₁₂₃₄,
 0.385 + 0.604 𝐞₁₂ + -0.613 𝐞₁₃ + -0.134 𝐞₁₄ + -0.582 𝐞₂₃ + -0.38 𝐞₂₄ + 0.256 𝐞₃₄,
 0.385 + 0.604 𝐞₁₂ + -0.613 𝐞₁₃ + -0.134 𝐞₁₄ + -0.582 𝐞₂₃ + -0.38 𝐞₂₄ + 0.256 𝐞₃₄,
 0.832 + 0.363 𝐞₁₂ + -0.368 𝐞₁₃ + -0.0806 𝐞₁₄ + -0.35 𝐞₂₃ + -0.228 𝐞₂₄ + 0.154 𝐞₃₄,
 0.832 + 0.363 𝐞₁₂ + -0.368 𝐞₁₃ + -0.0806 𝐞₁₄ + -0.35 𝐞₂₃ + -0.228 𝐞₂₄ + 0.154 𝐞₃₄,
 0.832 + 0.363 𝐞₁₂ + -0.368 𝐞₁₃ + -0.0806 𝐞₁₄ + -0.35 𝐞₂₃ + -0.228 𝐞₂₄ + 0.154 𝐞₃₄ + -2.08e-17 𝐞₁₂₃₄,
 0.832 + 0.363 𝐞₁₂ + -0.368 𝐞₁₃ + -0.0806 𝐞₁₄ + -0.35 𝐞₂₃ + -0.228 𝐞₂₄ + 0.154 𝐞₃₄ + 1.25e-16 𝐞₁₂₃₄)

In [54]:
#8.17b hyperbolic rotor (boost)
S = random_simple_rotor(alg, signature=1)
S2 = S ** 2
b = simple_rotor_log(S2)
S2, blade_exp(b), S, blade_exp(b/2), simple_rotor_sqrt(S2), blade_exp(3.3*b).sw(S)

(1.12 + -0.181 𝐞₁₂ + -0.0842 𝐞₁₃ + -0.275 𝐞₁₄ + 0.399 𝐞₂₃ + -0.0216 𝐞₂₄ + -0.616 𝐞₃₄ + -6.94e-18 𝐞₁₂₃₄,
 1.12 + -0.181 𝐞₁₂ + -0.0842 𝐞₁₃ + -0.275 𝐞₁₄ + 0.399 𝐞₂₃ + -0.0216 𝐞₂₄ + -0.616 𝐞₃₄,
 1.03 + -0.088 𝐞₁₂ + -0.0409 𝐞₁₃ + -0.133 𝐞₁₄ + 0.194 𝐞₂₃ + -0.0105 𝐞₂₄ + -0.299 𝐞₃₄,
 1.03 + -0.088 𝐞₁₂ + -0.0409 𝐞₁₃ + -0.133 𝐞₁₄ + 0.194 𝐞₂₃ + -0.0105 𝐞₂₄ + -0.299 𝐞₃₄,
 1.03 + -0.088 𝐞₁₂ + -0.0409 𝐞₁₃ + -0.133 𝐞₁₄ + 0.194 𝐞₂₃ + -0.0105 𝐞₂₄ + -0.299 𝐞₃₄ + -3.37e-18 𝐞₁₂₃₄,
 1.03 + -0.088 𝐞₁₂ + -0.0409 𝐞₁₃ + -0.133 𝐞₁₄ + 0.194 𝐞₂₃ + -0.0105 𝐞₂₄ + -0.299 𝐞₃₄ + 8.88e-16 𝐞₁₂₃₄)

In [55]:
#8.17c 8.20 parabolic rotor (translation)
S = (e2 + e3 - e4) * e2
S2 = S ** 2
b = simple_rotor_log(S2)
S2, blade_exp(b), S, blade_exp(b/2), simple_rotor_sqrt(S2), blade_exp(3.3*b).sw(S)

(1 + -2 𝐞₂₃ + 2 𝐞₂₄,
 1 + -2 𝐞₂₃ + 2 𝐞₂₄,
 1 + -1 𝐞₂₃ + 1 𝐞₂₄,
 1 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄,
 1.0 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄,
 1.0 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄)

In [56]:
#8.21
B = S.grade(2) / (1 + S.grade(0))
B, (1+B)**2/(normsq(1+B)), S

(-0.5 𝐞₂₃ + 0.5 𝐞₂₄, 1.0 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄, 1 + -1 𝐞₂₃ + 1 𝐞₂₄)

In [57]:
#8.25
R, u = random_rotor(alg), random_r_blade(1, alg)
v = R.sw(u)
S = simple_rotor_sqrt(v * u / normsq(u))
U = S.reverse() * R
R, S * U, S * u, u * S.reverse(), U * u, u * U

(-2.05 + 2.84 𝐞₁₂ + 0.989 𝐞₁₃ + 3.48 𝐞₁₄ + 1.15 𝐞₂₃ + 0.525 𝐞₂₄ + -1.05 𝐞₃₄ + -0.229 𝐞₁₂₃₄,
 -2.05 + 2.84 𝐞₁₂ + 0.989 𝐞₁₃ + 3.48 𝐞₁₄ + 1.15 𝐞₂₃ + 0.525 𝐞₂₄ + -1.05 𝐞₃₄ + -0.229 𝐞₁₂₃₄,
 1.61 𝐞₁ + 2.48 𝐞₂ + 0.92 𝐞₃ + 2.95 𝐞₄ + -2.22e-16 𝐞₁₂₃ + -2.78e-17 𝐞₁₂₄ + -2.22e-16 𝐞₁₃₄ + -2.78e-17 𝐞₂₃₄,
 1.61 𝐞₁ + 2.48 𝐞₂ + 0.92 𝐞₃ + 2.95 𝐞₄ + 2.22e-16 𝐞₁₂₃ + 2.22e-16 𝐞₁₃₄ + 2.78e-17 𝐞₂₃₄,
 -0.96 𝐞₁ + -0.0205 𝐞₂ + -0.573 𝐞₃ + -0.131 𝐞₄ + 1.32 𝐞₁₂₃ + 0.394 𝐞₁₂₄ + -1.4 𝐞₁₃₄ + -0.0839 𝐞₂₃₄,
 -0.96 𝐞₁ + -0.0205 𝐞₂ + -0.573 𝐞₃ + -0.131 𝐞₄ + 1.32 𝐞₁₂₃ + 0.394 𝐞₁₂₄ + -1.4 𝐞₁₃₄ + -0.0839 𝐞₂₃₄)

In [58]:
def rotor_split(R: MultiVector, alg: Algebra, tol=1e-6):
    # FIXME: when the tangent not well defined
    assert normsq(R.grade(2)) > tol, "zero bivector"
    tangent = R.grade(2) / R[0]
    tangents = bivector_split(tangent, alg)
    splits = sorted([normalize(1 + t) for t in tangents], key=lambda item: abs(item[0]))
    sign = np.round(gprod(r[0] for r in splits)/R[0])
    splits[0] = splits[0] * sign # to keep the sign of R
    return splits
        

R_splits = rotor_split(R, alg)
R, gprod(R_splits), gprod([blade_exp(simple_rotor_log(r)) for r in R_splits])

(-2.05 + 2.84 𝐞₁₂ + 0.989 𝐞₁₃ + 3.48 𝐞₁₄ + 1.15 𝐞₂₃ + 0.525 𝐞₂₄ + -1.05 𝐞₃₄ + -0.229 𝐞₁₂₃₄,
 -2.05 + 2.84 𝐞₁₂ + 0.989 𝐞₁₃ + 3.48 𝐞₁₄ + 1.15 𝐞₂₃ + 0.525 𝐞₂₄ + -1.05 𝐞₃₄ + -0.229 𝐞₁₂₃₄,
 -2.05 + 2.84 𝐞₁₂ + 0.989 𝐞₁₃ + 3.48 𝐞₁₄ + 1.15 𝐞₂₃ + 0.525 𝐞₂₄ + -1.05 𝐞₃₄ + -0.229 𝐞₁₂₃₄)

In [59]:
#8.10 a spinor is an even versor for odd dimension (a Dirac representation);
# a sum of an even versor and its dual for even dimension (two Weyl representations)
def random_study_number(alg):
    a, b = np.random.rand(2)
    return (a + b*alg.pseudoscalar((1,)))

def random_spinor(alg: Algebra):
    if alg.d % 4:
        return random_even_versor(alg)
    return random_study_number(alg) * random_even_versor(alg)

random_spinor(Algebra(3)), random_spinor(Algebra(4))


(0.0847 + -0.0432 𝐞₁₂ + 0.00199 𝐞₁₃ + 0.107 𝐞₂₃,
 0.713 + 0.111 𝐞₁₂ + 0.124 𝐞₁₃ + 0.25 𝐞₁₄ + -0.101 𝐞₂₃ + 0.295 𝐞₂₄ + 0.161 𝐞₃₄ + 0.133 𝐞₁₂₃₄)

In [60]:
# A pointor depends on a point
def random_point(alg: Algebra):
    return random_r_blade(alg.d - 1, alg)

O = random_point(alg)

# point reflection on the point itself is a pointor
terms_ratio(O, O.sw(O))

array([-33.57429333, -33.57429333, -33.57429333, -33.57429333])

In [61]:
# versors not crossing O-subspace are pointers 
vectors = blade_split(O, alg)
B1, B2 = wedge(vectors[:2]), wedge(vectors[1:])
Od = O.dual()
terms_ratio(O, Od.sw(O)), terms_ratio(B1.sw(O), O), terms_ratio(vectors[0].sw(O), O), 

(array([-33.57429333, -33.57429333, -33.57429333, -33.57429333]),
 array([12.90591293, 12.90591293, 12.90591293, 12.90591293]),
 array([13.36279296, 13.36279296, 13.36279296, 13.36279296]))

In [62]:
# rotors with reflections are pointers
R1 = blade_exp(3.3*B1)
R2 = blade_exp(4.4*B2)
terms_ratio((Od * R1 * R2).sw(O), O), terms_ratio((Od * vectors[2] * R1).sw(O), O)

(array([-0.02978469, -0.02978469, -0.02978469, -0.02978469]),
 array([0.03264996, 0.03264996, 0.03264996, 0.03264996]))

In [63]:
# rotors cross O-subspace may not be pointers
terms_ratio(blade_exp(Od ^ vectors[2]).sw(O), O)

array([2.31265738, 1.17541562, 1.17541562, 1.17541562])

In [64]:
# A general pointor suggested by "From Invariant Decomposition to Spinors"?
# How do we find the point specified by a pointor?
def random_odd_versor(alg):
    d = alg.d
    if not (alg.d % 2):
        d -= 1
    return gprod(random_r_vectors(d, alg))


# Actually I'm not sure these generate pointors
# def random_pointor(alg: Algebra):
#     return random_odd_versor(alg) * random_even_versor(alg)
    
# def random_pointor(alg: Algebra):
#     v = random_r_blade(1, alg)
#     return (np.random.rand() + v) * random_even_versor(alg)

# FIXME: check degree of freedom
def random_pointor(alg: Algebra):
    v = random_r_blade(1, alg)
    return (np.random.rand() + v) * random_versor(2, alg)

pointor = random_pointor(alg)
pointor

0.485 + 0.228 𝐞₁ + 0.447 𝐞₂ + 0.646 𝐞₃ + 0.0591 𝐞₄ + -0.0795 𝐞₁₂ + -0.0623 𝐞₁₃ + -0.0573 𝐞₁₄ + 0.116 𝐞₂₃ + -0.032 𝐞₂₄ + -0.108 𝐞₃₄ + 0.00599 𝐞₁₂₃ + 0.0281 𝐞₁₂₄ + 0.0177 𝐞₁₃₄ + -0.0433 𝐞₂₃₄

In [65]:
# the point-grade
O = pointor.grade(3)
terms_ratio(pointor.sw(O), O)

array([0.90449692, 0.90449692, 0.90449692, 0.90449692])